# Random Forest Classification

Importing libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

Loading the dataset

In [3]:
df = pd.read_csv("D:\Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.shape

(10000, 14)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [8]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

There is no null values in the dataset. I'll just skip to dropping the unnecessary columns.

In [9]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Feature Engineering

Transforming categorical data into numerical data using One hot encoding

In [10]:
df.Gender.unique()

array(['Female', 'Male'], dtype=object)

In [20]:
pd.get_dummies(df['Gender']).head()

,Female,Male
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [21]:
Gender = pd.get_dummies(df['Gender'],drop_first=True) 
Gender.head()

,Male
0,0
1,0
2,0
3,0
4,0


In [22]:
Gender = pd.get_dummies(df['Gender'],drop_first=True) 
Gender.head()

,Male
0,0
1,0
2,0
3,0
4,0


In [23]:
df.Geography.unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [24]:
pd.get_dummies(df['Geography']).head()

,France,Germany,Spain
0,1,0,0
1,0,0,1
2,1,0,0
3,1,0,0
4,0,0,1


In [26]:
Geography = pd.get_dummies(df['Geography'],drop_first=True) 
Geography.head()

,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1


Concatinating back to dataframe

In [28]:
new_data = pd.concat([df,Gender,Geography],axis=1)
new_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,Germany,Spain
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,0,1


Dropping redundant columns

In [29]:
new_data.drop(['Gender','Geography'],axis=1,inplace=True)
new_data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Male,Germany,Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1


Creating separate target variable

In [30]:
Y=new_data['Exited']
Y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [31]:
Y.unique()

array([1, 0], dtype=int64)

Droping the target variable from the dataset

In [32]:
X=new_data.drop(['Exited'],axis=1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Male,Germany,Spain
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1


Splittin the dataset into training and testing set

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [34]:
print('X_train',X_train.shape)
print('X_test',X_test.shape)
print('Y_train',Y_train.shape)
print('Y_test',Y_test.shape)


X_train (8000, 11)
X_test (2000, 11)
Y_train (8000,)
Y_test (2000,)


Training Random Forest Classification model on the Training set

In [36]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=5,criterion='entropy',random_state=0)
classifier.fit(X_train,Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=5, random_state=0)

Predicting the Test set results

In [37]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

Confusion Matrix

In [38]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,classification_report

In [39]:
cm = confusion_matrix(Y_test,y_pred)
print(cm)

[[1484  111]
 [ 192  213]]


In [40]:
accuracy_score(Y_test,y_pred)

0.8485

In [41]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91      1595
           1       0.66      0.53      0.58       405

    accuracy                           0.85      2000
   macro avg       0.77      0.73      0.75      2000
weighted avg       0.84      0.85      0.84      2000



Trying with different number of trees and gini criteria

In [44]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=12,criterion='gini',random_state=0)
classifier.fit(X_train,Y_train)


RandomForestClassifier(n_estimators=12, random_state=0)

In [45]:
y_pred = classifier.predict(X_test)

In [46]:
accuracy_score(Y_test,y_pred)

0.8585

with more number of trees and gini criteria, the accuracy is increased